In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import math
import ROOT
import numpy as np, pandas as pd
import yaml
import base64
import ctypes
import logging
import glob
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.Tools.compressArray import arrayCompressionRelative8
from TPCQCVis.src.palette import *
#output_notebook()

In [ ]:
#inputFile="/lustre/alice/users/miranov/NOTESData/alice-tpc-notes/JIRA/ATO-592/tpcqcskimmingTracks.root"
#inputFile="~/Software/TPCQCVis/data/tpcqcskimmingTracks.root"
path = "/home/berki/Software/TPCQCVis/data/"
fileList = glob.glob(path+"/skimmed_LHC22c_*_001.root")
fileList.sort()
runList = [fileList[i][len(path)+15:-9] for i in range(len(fileList))]

In [ ]:
df = pd.DataFrame()
i=0
for f in fileList:
    f=ROOT.TFile.Open(f)
    tracks = f.Get("tracks")
    tracks.SetAlias("weight1","((triggerMask&0x1)>0)*100*max(weight,0.01)")
    tracks.SetAlias("weight2","((triggerMask&0x2)>0)*100*max(weight*pt*pt,0.01)")
    tracks.SetAlias("weight3","((triggerMask&0x4)>0)*100")
    tracks.SetAlias("weight_full","weight1+weight2+weight3")
    tracks.SetAlias("rMin","trackPar.mX")
    tracks.SetAlias("qPt","trackPar.mP[4]")
    tracks.SetAlias("pzPt","trackPar.mP[3]")
    tracks.SetAlias("itsOn","itsClusterMap>0")
    tracks.SetAlias("tofOn","abs(tofChi2)<100")
    tracks.SetAlias("p","pt*sqrt(1+pzPt*pzPt)")
    tracks.SetAlias("logp","log(p)")
    #tracks.SetAlias("logtpcSignal","log(tpcSignal)")
    tracks.SetAlias("eta","trackPar.getEta()")
    tracks.SetAlias("theta","trackPar.getTheta()")
    tracks.SetAlias("tgl","trackPar.getTgl()")
    tracks.SetAlias("snp","trackPar.getSnp()")
    df_tmp=tree2Panda(tracks, [".*"], "", exclude=["trackPar"])
    df_tmp["dcaDefined"]=df_tmp.eval("dcaXY!=0")
    df_tmp["side"]=df_tmp.eval("pzPt>0") ##  (A/C/CrossAC/CrossCA side)
    df_tmp["isPrim5"]=df_tmp.eval("abs(dcaXY)<5 &  abs(dcaZ)<5 & rMin<5")
    #df_tmp["tpcNCR"]=df_tmp["tpcNClsFindable"]-df_tmp["tpcNClsFindableMinusCrossedRows"]
    # deleting unused columns
    df_tmp.drop("tofExpMom", inplace=True, axis=1)
    df_tmp.drop("itsChi2NCl", inplace=True, axis=1)
    df_tmp.drop("trdPattern", inplace=True, axis=1)
    df_tmp.drop("vertex_", inplace=True, axis=1)
    df_tmp.drop("trackEtaEmcal", inplace=True, axis=1)
    df_tmp.drop("tpcNClsShared", inplace=True, axis=1)
    df_tmp.drop("flags", inplace=True, axis=1)
    df_tmp.drop("tpcNClsFindableMinusFound", inplace=True, axis=1)
    df_tmp.drop("trdChi2", inplace=True, axis=1)
    df_tmp.drop("trackPhiEmcal", inplace=True, axis=1)
    df_tmp.drop("tofChi2", inplace=True, axis=1)
    df_tmp["runNumber"]=runList[i]
    df=pd.concat([df,df_tmp], ignore_index=True)
    i=i+1

In [ ]:
df.head()

In [ ]:
parameterArray = [
    # scatter
    {"name": "size", "value":5, "range":[0, 20]},
    # legend
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
    {"name": "legendVisible", "value":True},
    {"name": "legendLocation", "value":"top_right", "options":["top_right","top_left", "bottom_right","bottom_left"]},
    # axis transform
    {"name": "zAxisTrans", "value":"lin_z", "options":["lin_z", "log_z","sqrt_z"]},
    {"name": "xAxisTrans_0", "value":"lin_x_0", "options":["lin_x_0", "log_x_0","sqrt_x_0"]},
    # skimming
    {"name": "weighting", "value":"weight_full", "options":["weight_full","weight1", "weight2","weight3"]},
]

In [ ]:
widgetParams=[
      # Geometry
      ['range', ['phi']],
      ['range', ['eta']],
      ['range', ['theta']],
      ['range', ['tgl']],
      ['multiSelect',["side"]],
      # Momenta
      ['range', ['pt']],
      ['range', ['qPt']],
      ['range', ['pzPt']],
      ['range', ['p']],
      # vertex
      ['range', ['dcaXY']],
      ['range', ['dcaZ']],
      ['range', ['rMin']],
      ['multiSelect',["dcaDefined"]],
      ['multiSelect',["isPrim5"]],
      ['multiSelect',["hasCollision"]],
      # detectors
      ['range', ['tpcNClsFindable']],
      ['range', ['tpcSignal']],
      ['multiSelect',["itsOn"]],
      ['multiSelect',["tofOn"]],
      # other
      ['textQuery', {"title": "user select"}],
      # skimming
      ['multiSelect',["triggerMask"]],
      ['select',["weighting"]],
      # Graphics
      ['select',["zAxisTrans"], {"callback": "parameter", "default": 0, "title":"z-Axis transformation"}],
      ['slider',["size"], {"callback": "parameter", "title":"Marker size"}],
      ['toggle',['legendVisible']],  
      ['select',["legendFontSize"], {"callback": "parameter", "default": 2, "title":"Legend font size"}],
      ['select',["legendLocation"], {"callback": "parameter", "default": 0}],
      #Runs
      ['multiSelect',["runNumber"]],
]
widgetLayoutDesc={
    "Geometry": [[0,1,2,3],[4], {'sizing_mode': 'scale_width'}],
    "Momentum": [[5,6],[7,8], {'sizing_mode': 'scale_width'}],
    "Vertex": [[9,10,11],[12,13,14], {'sizing_mode': 'scale_width'}],
    "Detectors": [[15,16],[17,18], {'sizing_mode': 'scale_width'}],
    "UserSelect": [[19], {'sizing_mode': 'scale_width'}],
    "Skimming": [[20,21], {'sizing_mode': 'scale_width'}],
    "Graphics": [[22,23],[24,25,26], {'sizing_mode': 'scale_width'}],
    "Run": [27]
}

In [ ]:
histoArray = [
    {"name": "pT_hist", "variables": ["pt"], "nbins":100, "range": [0,20],"weights":"weighting"},
    {"name": "tpcSignal_hist", "variables": ["tpcSignal"], "nbins":100, "range": [0,None],"weights":"weighting"},
]
histoArrayTPCsignal=[
    {"name": "tpcSignalVslogp", "variables": ["logp","tpcSignal"], "nbins":[100,100], "range": [[math.log(0.05),math.log(20)],[0, 500]],"weights":"weighting"},
    {"name": "tpcSignalVsqPt", "variables": ["qPt","tpcSignal"], "nbins":[100,100], "range": [[-20,20],[0, 500]],"weights":"weighting"},
    {"name": "tpcSignalVsphi", "variables": ["phi","tpcSignal"], "nbins":[90,100], "range": [[0, 6.28],[0, 500]],"weights":"weighting"},
    {"name": "tpcSignalVstpcNClsFindable", "variables": ["tpcNClsFindable","tpcSignal"], "nbins":[85,100], "range": [[0, 170],[0, 500]],"weights":"weighting"},
    {"name": "tpcSignalVseta", "variables": ["eta","tpcSignal"], "nbins":[40,100], "range": [[-2, 2],[0, 500]],"weights":"weighting"},
    {"name": "tpcSignalVstgl", "variables": ["tgl","tpcSignal"], "nbins":[40,100], "range": [[-2, 2],[0, 500]],"weights":"weighting"}
]
histoArray.extend(histoArrayTPCsignal)

histoArrayPt=[
    {"name": "ptVsphi", "variables": ["phi","pt"], "nbins":[90,100], "range": [[0, 6.28],[0,20]],"weights":"weighting"},
    {"name": "ptVstpcNClsFindable", "variables": ["tpcNClsFindable","pt"], "nbins":[80,100], "range": [[0,160],[0,20]],"weights":"weighting"},
    {"name": "ptVseta", "variables": ["eta","pt"], "nbins":[40,100], "range": [[-2, 2],[0, 20]],"weights":"weighting"},
    {"name": "ptVstgl", "variables": ["tgl","pt"], "nbins":[40,100], "range": [[-2, 2],[0, 20]],"weights":"weighting"}
]
histoArray.extend(histoArrayPt)

histoArrayNCluster=[
    {"name": "tpcNClsFindable_Hist", "variables": ["tpcNClsFindable"], "nbins":32, "range": [0,160],"weights":"weighting"},
    {"name": "tpcNClsFindableVsphi", "variables": ["phi","tpcNClsFindable"], "nbins":[90,100], "range": [[0, 6.28],[0,160]],"weights":"weighting"},
    {"name": "tpcNClsFindableVseta", "variables": ["eta","tpcNClsFindable"], "nbins":[40,100], "range": [[-2, 2],[0,160]],"weights":"weighting"},
    {"name": "tpcNClsFindableVstgl", "variables": ["tgl","tpcNClsFindable"], "nbins":[40,100], "range": [[-2, 2],[0,160]],"weights":"weighting"}
]
histoArray.extend(histoArrayNCluster)


In [ ]:
aliasArray = []
histoArrayLog=[]
for his in histoArray:
    histoName=his["name"]
    # z (color)
    histoArrayLog.append({ "name": f"lin_z","variables": ["bin_count"],"func": "return Math.max(bin_count,0)","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"log_z","variables": ["bin_count"],"func": "return Math.log(Math.max(bin_count,1))","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"sqrt_z","variables": ["bin_count"],"func": "return Math.sqrt(Math.max(bin_count,0))","context": f"{histoName}"})
aliasArray.extend(histoArrayLog)

In [ ]:
figureArray = [
    #1Ds
    [['bin_center'],['bin_count'],{"source":"pT_hist","colorZvar":"runNumber","xAxisTitle":"log(p)", "yAxisTitle":"tpcSignal"}],
    # tpcSignal
    [['bin_center_0'],['bin_center_1'],{"source":"tpcSignalVslogp","colorZvar":"zAxisTrans","xAxisTitle":"log(p)", "yAxisTitle":"tpcSignal"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcSignalVsqPt","colorZvar":"zAxisTrans", "xAxisTitle":"q/pT", "yAxisTitle":"tpcSignal"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcSignalVsphi","colorZvar":"zAxisTrans", "xAxisTitle":"phi", "yAxisTitle":"tpcSignal"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcSignalVstpcNClsFindable","colorZvar":"zAxisTrans", "xAxisTitle":"tpcNClsFindable", "yAxisTitle":"tpcSignal"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcSignalVseta","colorZvar":"zAxisTrans", "xAxisTitle":"eta", "yAxisTitle":"tpcSignal"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcSignalVstgl","colorZvar":"zAxisTrans", "xAxisTitle":"tan(lambda)", "yAxisTitle":"tpcSignal"}],
    # pT
    [['bin_center_0'],['bin_center_1'],{"source":"ptVsphi","colorZvar":"zAxisTrans", "xAxisTitle":"phi", "yAxisTitle":"pt"}],
    [['bin_center_0'],['bin_center_1'],{"source":"ptVstpcNClsFindable","colorZvar":"zAxisTrans", "xAxisTitle":"tpcNClsFindable", "yAxisTitle":"pt"}],
    [['bin_center_0'],['bin_center_1'],{"source":"ptVseta","colorZvar":"zAxisTrans", "xAxisTitle":"eta", "yAxisTitle":"pt"}],
    [['bin_center_0'],['bin_center_1'],{"source":"ptVstgl","colorZvar":"zAxisTrans", "xAxisTitle":"tan(lambda)", "yAxisTitle":"pt"}],
    # nClustersFindable
    [['tpcNClsFindable'],['tpcNClsFindable_Hist'],{"xAxisTitle":"tpcNClsFindable", "yAxisTitle":"counts","size":"size","visualization_type":"bars","color":"#AAAAAA"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcNClsFindableVsphi","colorZvar":"zAxisTrans", "xAxisTitle":"phi", "yAxisTitle":"tpcNClsFindable"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcNClsFindableVseta","colorZvar":"zAxisTrans", "xAxisTitle":"eta", "yAxisTitle":"tpcNClsFindable"}],
    [['bin_center_0'],['bin_center_1'],{"source":"tpcNClsFindableVstgl","colorZvar":"zAxisTrans", "xAxisTitle":"tan(lambda)", "yAxisTitle":"tpcNClsFindable"}],
    
    {'plot_height':150, "commonX":0,"palette":kBird9,"visualization_type":"heatmap",
     "legend_options": {"label_text_font_size": "legendFontSize", "visible": "legendVisible", "location":"legendLocation"}}
]            

In [ ]:
figureLayout = {
    "TPC Signal": {
        "p & qPt": [[0,1], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}],
        "phi & tpcNClsFindable": [[2,3], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}],
        "eta & tan(lambda)": [[4,5], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}]
    },
    "pT": {
        "phi & tpcNClsFindable": [[6,7], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}],
        "eta & tan(lambda)": [[8,9], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}]
    },
    "nClustersFindable": {
        "1D & phi": [[10,11], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}],
        "eta & tan(lambda)": [[12,13], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}]
    }
}

In [ ]:
tooltips=[]
output_file("tpcQCdashboard.html")
xxx = bokehDrawSA.fromArray(df, "pt<20&abs(qPt)<20&abs(pzPt)<2", figureArray, widgetParams, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                            parameterArray=parameterArray, histogramArray=histoArray,aliasArray=aliasArray,sizing_mode='scale_width',
                            arrayCompression=arrayCompressionRelative8)